## Coursera - IBM Data Science Capstone Project ##

### Brian Lee ###
BrianTrewLee@gmail.com<br />
LinkedIn: https://www.linkedin.com/in/brian-lee-64a2022b/



## 1. Introduction ##

If I were to open a Starbucks, where should I locate it? Clearly, the location is one of the most important business decisions for this venture. Is there a science to locating a StarBucks? Can we create a machine learning model to predict where a StarBucks is likely to be located? I'd like to find out.

If a reliable model can be made,it could be used in the process of opening a store. It could be used as a final sanity check, or at the beginning stage, to select a promising location. If a trustworthy model predicts with a high degree of confidence that a Starbucks should be located in an area, but there is not one there yet, perhaps that is an opportunity.

Any person interested in opening a Starbucks should be interested in these results. This includes myself, other potential franchisees, and the Starbucks corporation itself, which operates many of it's own stores. The Starbucks competition might also be interested as they could possibly gain competitive insigths. I also believe others might be interested in this procedure, as it might be applied to predicting the location of other entities, based on the same sort of data.

The main purpose of this project is to prove the concept of predicting Starbuck's locations in general. I suspect it may work better or worse depending on the locations chosen for training and prediction, the radius size, and the specific features used in the model. I may vary those factors in order to prove the concept, which could then be applied carefully to a particular geographic area of interest at a later date.

### 2. Data ###

Foursquare (https://foursquare.com) is a company that crowdsources location data, tying latitude-longitude coordinates (and other things) with public venues, including many businesses, such as Starbucks locations. The features for the Starbucks location classifier will come from venue data from the Fourquare API. I will use venue category names (such as 'bar', 'Chinese restaurant', 'coffee shop') as features to classify an area as either an area with a Starbucks, or an area without one. I will use the venue name (e.g. 'Starbucks') to determine whether or not the venue is a Starbucks. An area containing a Starbucks contains at least one venue named 'Starbucks' within the radius supplied to the Foursquare API's explore' endpoint.

An area will be a circular area with a given radius. I have not determined that radius yet, and I might use more than one radius in this project. Within that radius, the Foursquare API will return venues. The larger the radius, the more venues that could be anticipated, and the higher likelihood of a Starbucks, but a larger radius is also less useful for business use. I have done some exploratory work with a radius of 300m.

The areas to be chosen should contain a mix of areas with Starbuck's and those without. I have done some exploratory work in a particular area of San Francisco. I located Starbucks with Google Maps and got latitude and longitude points making a box around that area. I could choose the train and test points at random within an area, or I could generate a grid covering an entire area. Given the constraints my Foursquare Developer account places on my searches, and knowing that binary classifiers work better with a good number of examples of both classes, I may want to generate areas that I know will be heavy with Starbucks locations. To that end, I discovered a Kaggle dataset (https://www.kaggle.com/starbucks/store-locations) containing the latitude and longitude coordinates of 25,600 worldwide Starbucks locations. I may or may not use that resource.

### 3. Methodology ###

I will generate a corpus of geographic points, and using a reasonable radius (perhaps 300m) a corpus of geographic areas. Each geographic area will have a feature set based on json data returned by the Foursquare API's 'explore' endpoint. I plan to use the category names of the returned venues as features for that area. I will use the venue names to determine whether oir not a Starbucks is in that area, and generate the labels from that. 

Once I have labelled data, I will use sklearn.model_selection.train_test_split to generate tarining and testing sets. Then I  can use any of a host of sklearn classifiers to generate predictive models. I have done some exploratory work with a random forest.

In [2]:
# import libraries

import urllib.request # open and read URLs

import json # handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # handle requests
import pandas as pd # process data as dataframes with Pandas
import numpy as np # handle data in a vectorized manner with NumPy

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't installed the GeoPy geocoding library yet
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

##!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't installed the Folium library yet
import folium # map rendering library

# Matplolib plotting library and associated modules
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # for K-Means clustering with Scikit-Learn

print("Libraries imported!")

Libraries imported!


In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
# Define a function to gather recommended venues, with specifically the name and category, using the explore API

def getNearbyVenues(latitudes, longitudes, radius=300, limit=100):
    
    count = 0
    venues_list = []
    for lat, lng in zip(latitudes, longitudes):
        #create a unique namefor this pointthat is easier todeal with than a combination of lat and lon
        point_name = "point"+str(count)
        print(point_name,':',lat,'-',lng)
         

            
        # create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        #print("resultscols=",results.columns)
        

        
        # return only relevant information for each nearby venue
        venues_list.append([(
            point_name,
            lat, 
            lng, 
            v["venue"]["name"], 
            v["venue"]["location"]["lat"], 
            v["venue"]["location"]["lng"],  
            v["venue"]["location"]["distance"], 
            v["venue"]["categories"][0]["name"]) for v in results])
        count = count + 1
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Point Name",
                  "Neighborhood Latitude", 
                  "Neighborhood Longitude", 
                  "Venue", 
                  "Venue Latitude", 
                  "Venue Longitude", 
                  "Venue Distance",           
                  "Venue Category"]
    
    return(nearby_venues)

In [5]:
sf_lat=37.7749
sf_lon=-122.4194
ch_lat=41.8781
ch_lon=-87.6298
la_lat=34.0522
la_lon=-118.2437
starbucks_m_lat=37.744644
starbucks_m_lon=-122.452731
import random

#37.780253, -122.509117
#37.804203, -122.459264 - upper left of starbucks intensive space
#37.767547, -122.409342 - lower right ofstarbucksintensive space
llat=37.767547
ulat=37.804203
llon=-122.459264
ulon=-122.409342
random_lat0 = random.uniform(37.767547,37.804203)
random_lon0 = random.uniform(-122.459264, -122.409342)
random_lat1 = random.uniform(llat,ulat)
random_lon1 = random.uniform(llon, ulon)
random_lat2 = random.uniform(llat,ulat)
random_lon2 = random.uniform(llon, ulon)
random_lat3 = random.uniform(llat,ulat)
random_lon3 = random.uniform(llon, ulon)
random_lat4 = random.uniform(llat,ulat)
random_lon4 = random.uniform(llon, ulon)
random_lat5 = random.uniform(llat,ulat)
random_lon5 = random.uniform(llon, ulon)
random_lat6 = random.uniform(llat,ulat)
random_lon6 = random.uniform(llon, ulon)
random_lat7 = random.uniform(llat,ulat)
random_lon7 = random.uniform(llon, ulon)
random_lat8 = random.uniform(llat,ulat)
random_lon8 = random.uniform(llon, ulon)
random_lat9 = random.uniform(llat,ulat)
random_lon9 = random.uniform(llon, ulon)

data = pd.DataFrame(np.array([[sf_lat, sf_lon], [ch_lat, ch_lon], [la_lat, la_lon], [starbucks_m_lat,starbucks_m_lon], [random_lat0,random_lon0], [random_lat1,random_lon1], [random_lat2,random_lon2], [random_lat3,random_lon3], [random_lat4,random_lon4], [random_lat5,random_lon5], [random_lat6,random_lon6], [random_lat7,random_lon7], [random_lat8,random_lon8], [random_lat9,random_lon9] ]), columns=['Latitude', 'Longitude'])

data



,Latitude,Longitude
0,37.774900,-122.419400
1,41.878100,-87.629800
2,34.052200,-118.243700
3,37.744644,-122.452731
4,37.799287,-122.439278
5,37.799989,-122.420869
6,37.792190,-122.414503
7,37.800762,-122.456176
8,37.769161,-122.444804
9,37.782793,-122.410721


In [6]:
#getrid#random_latlons = []
#getrid#for j in range(0,250):
#getrid#    random_lat = random.uniform(llat,ulat)
#getrid#    random_lon = random.uniform(llon,ulon)
#getrid#    random_latlon=(random_lat,random_lon)
#getrid#    random_latlons.append(random_latlon)
    

In [7]:
#getrid#random_latlons = pd.DataFrame(random_latlons)
#getrid#random_lats=random_latlons[0]
#getrid#random_lons=random_latlons[1]
#getrid#random_lats.shape



(250,)

In [8]:
#getrid#random_latlons[0].shape

(250,)

In [9]:
#getrid#random_latlons[1].shape

(250,)

In [10]:


#getrid#data["Latitude"] = random_latlons[0]
#getrid#data["Longitude"] = random_latlons[1]

#getrid#data["Latitude"].head()
#getrid#data["Latitude"].shape


(14,)

In [11]:
#37.599350, -122.496830
#37.807957, -122.390854
#reset parameters to bigger area
##llat = 37.599350
#ulat = 37.807957
##llon = -122.496830
##ulon = -122.390854


points = []
radius = 500 #in meters
current_lat = ulat
current_lon = llon
import geopy
from geopy.distance import VincentyDistance
d = radius/1000 #in kilometers
d=d*2
b=90 #in degrees 90 = east,180=south,270=west
# given: lat1, lon1, b = bearing in degrees, d = distance in kilometers

def lon_pass(current_lat, current_lon, ulat, ulon):
    while (current_lat <= ulat) and (current_lon <= ulon) and (current_lat >= llat) and (current_lon >= llon):
        origin = geopy.Point(current_lat, current_lon)
        destination = VincentyDistance(kilometers=d).destination(origin, b)

        current_lat, current_lon = destination.latitude, destination.longitude
        print(current_lat, current_lon)
        points.append((current_lat, current_lon))

while (current_lat <= ulat) and (current_lon <= ulon) and (current_lat >= llat) and (current_lon >= llon):
    print('first line of while')
    lon_pass(current_lat, current_lon, ulat, ulon)
    print('new row of coordinates')  
    current_lon=llon#move back to beginning of row   
    origin = geopy.Point(current_lat, current_lon)
    destination = VincentyDistance(kilometers=d).destination(origin, 180)#move south one step for next row
    current_lat, current_lon = destination.latitude, destination.longitude

    
    


first line of while
37.80420245275432 -122.44790881640202
37.804201905508656 -122.43655363288781
37.804201358263 -122.4251984494574
37.80420081101736 -122.41384326611077
37.804200263771726 -122.40248808284794
new row of coordinates
first line of while
37.795192848169535 -122.44791019552734
37.79519230110041 -122.43655639113842
37.79519175403129 -122.42520258683321
37.79519120696219 -122.41384878261171
37.795190659893095 -122.40249497847394
new row of coordinates
first line of while
37.78618322976931 -122.44791157403824
37.786182682876685 -122.43655914816013
37.78618213598408 -122.42520672236566
37.78618158909147 -122.41385429665486
37.786181042198876 -122.40250187102771
new row of coordinates
first line of while
37.77717359755475 -122.44791295193495
37.77717305083859 -122.43656190395349
37.77717250412243 -122.42521085605561
37.77717195740628 -122.41385980824134
37.777171410690144 -122.40250876051066
new row of coordinates
first line of while
37.76816395152695 -122.44791432921781
37.768

#### We many need more examples of the near-Starbucks class for a balanced set - so get a bunch of points we know are near to Starbucks
I pulled coordinates from the Kaggle Starbucks locations dataset<br />
The precision of the Kaggle set wasn't good enough to ensure the Starbucks location would be within the radius chosen, so I found a different dataset with higher precision

In [12]:
#starbucks_heavy_coordinates = pd.read_csv('data/directory-us-ca.csv') #this file gave lat-lon tothe hundreths, but that isn't good enough to ensure a Starbucks is within my radius
starbucks_heavy_coordinates = pd.read_csv('data/starbucks_us_ca_locations.csv',header=None)
starbucks_heavy_coordinates.columns = ["Longitude", "Latitude", "Location", "Address"]


In [13]:
starbucks_heavy_coordinates

,Longitude,Latitude,Location,Address
0,-118.114954,34.079000,Starbucks - CA - ALHAMBRA [W] 00373,"Valley & Almansor, Alhambra_810 E VALLEY BLVD_..."
1,-117.428300,34.626700,Starbucks - CA - Adelanto [WD] 00374,"Palmdale & Hwy 395, Adelanto_14136 US Hwy 395_..."
2,-118.757319,34.153435,Starbucks - CA - Agoura Hills 00375,Vons-Agoura Hills #2001_5671 Kanan Rd._Agoura ...
3,-118.757134,34.160600,Starbucks - CA - Agoura [W] 00376,"Agoura_5827 Canan Road_Agoura, California 9130..."
4,-122.277604,37.788222,Starbucks - CA - Alameda 00377,Albertsons-Alameda #7031_815 Marina Village_Al...
5,-122.253388,37.757099,Starbucks - CA - Alameda 00378,Safeway - Alameda #2708_2227 South Shore Cente...
6,-122.233245,37.768070,Starbucks - CA - Alameda 00379,Tilden & Blanding - Alameda_2671 Blanding Aven...
7,-122.243242,37.763696,Starbucks - CA - Alameda [W] 00380,Park Street & Central - Alameda_1364 Park Stre...
8,-122.252229,37.756650,Starbucks - CA - Alameda [W] 00381,South Shore Shopping Center_2210-J South Shore...
9,-122.276112,37.779659,Starbucks - CA - Alameda [W] 00382,Webster & Atlantic - Alameda_720 Atlantic Aven...


In [14]:
num_needed_to_balance = len(points)
num_needed_to_balance

25

In [15]:
starbucks_heavy_coordinates['Latitude'][1]  

34.6267

In [16]:
shc_lat, shc_lon = starbucks_heavy_coordinates.Latitude, starbucks_heavy_coordinates.Longitude

nnl=(shc_lat[0:num_needed_to_balance],shc_lon[0:num_needed_to_balance])
pnnl = pd.DataFrame(nnl)
pnnl.shape
pnnl = pnnl.transpose()
pnnl
newlats=pnnl['Latitude']
newlons=pnnl['Longitude']
newlats
points = pd.DataFrame(points)
current_end = len(points)
for jjj in range(0,num_needed_to_balance):
    print(str(jjj))
    lenp = len(points)
    points = points.set_value(lenp, 0, newlats[jjj])
    points = points.set_value(lenp, 1, newlons[jjj])  

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,0,1
0,37.804202,-122.447909
1,37.804202,-122.436554
2,37.804201,-122.425198
3,37.804201,-122.413843
4,37.804200,-122.402488
5,37.795193,-122.447910
6,37.795192,-122.436556
7,37.795192,-122.425203
8,37.795191,-122.413849
9,37.795191,-122.402495


In [17]:
points.shape

(50, 2)

shuffle the points

In [18]:
points = points.sample(frac=1).reset_index(drop=True)
points.head()

,0,1
0,33.868545,-117.751970
1,38.165852,-122.263920
2,37.757099,-122.253388
3,37.779659,-122.276112
4,38.171000,-122.245290


In [19]:
points.tail()

,0,1
45,37.777173,-122.436562
46,34.094562,-118.127970
47,37.849275,-122.031998
48,33.851333,-117.792281
49,37.763696,-122.243242


In [20]:
points

,0,1
0,33.868545,-117.751970
1,38.165852,-122.263920
2,37.757099,-122.253388
3,37.779659,-122.276112
4,38.171000,-122.245290
5,37.804202,-122.447909
6,37.768163,-122.436565
7,37.756650,-122.252229
8,37.777171,-122.402509
9,34.626700,-117.428300


In [21]:
map = folium.Map(location=[ulat, ulon], zoom_start=10)


# add markers to map
for lat, lng in zip(points[0],points[1]):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
map

In [22]:
venues = getNearbyVenues(latitudes = points[0],
                              longitudes =  points[1]
                               )


point0 : 33.86854485 - -117.7519695
point1 : 38.165852 - -122.26392
point2 : 37.757099 - -122.25338799999999
point3 : 37.77965876 - -122.2761122
point4 : 38.171 - -122.24529
point5 : 37.80420245275432 - -122.44790881640202
point6 : 37.7681634049872 - -122.43656465851913
point7 : 37.75665 - -122.252229
point8 : 37.777171410690144 - -122.40250876051066
point9 : 34.6267 - -117.4283
point10 : 34.079716 - -118.151899
point11 : 37.804200263771726 - -122.40248808284794
point12 : 37.768161765368006 - -122.40251564692426
point13 : 34.1606 - -118.75713400000001
point14 : 37.79519175403129 - -122.42520258683321
point15 : 37.88531 - -122.30793
point16 : 37.78618213598408 - -122.42520672236566
point17 : 37.788222 - -122.277604
point18 : 33.79959067 - -117.91904960000001
point19 : 33.836449 - -117.914482
point20 : 37.786182682876685 - -122.43655914816013
point21 : 37.78618158909147 - -122.41385429665486
point22 : 37.76807 - -122.233245
point23 : 34.079 - -118.114954
point24 : 37.78618322976931 - -12

In [23]:
venues.tail()

,Point Name,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
1451,point49,37.763696,-122.243242,Project Burger,37.764498,-122.243350,89,Burger Joint
1452,point49,37.763696,-122.243242,Afghan Tandoor,37.764692,-122.243710,118,Food
1453,point49,37.763696,-122.243242,Alameda Ballet Academy,37.764333,-122.242267,111,Dance Studio
1454,point49,37.763696,-122.243242,Cafe Central,37.764633,-122.243988,123,Deli / Bodega
1455,point49,37.763696,-122.243242,Kofman Auditorium,37.765379,-122.245716,287,Performing Arts Venue


In [24]:
venues['Venue'].unique()
venues['Venue'].value_counts()

Starbucks                                       29
Subway                                           9
Chase Bank                                       7
Peet's Coffee & Tea                              7
Taco Bell                                        6
Wells Fargo                                      6
Panera Bread                                     5
GNC                                              5
CVS pharmacy                                     5
Redbox                                           5
Jamba Juice                                      4
VONS                                             4
Shell                                            4
FedEx Office Print & Ship Center                 4
See's Candies                                    4
Rite Aid                                         4
The UPS Store                                    4
Trader Joe's                                     4
Safeway                                          4
AT&T                           

In [25]:
venues[350:400]


# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['point_name'] = venues['Point Name'] 
onehot['point_lat'] = venues['Neighborhood Latitude'] 
onehot['point_lon'] = venues['Neighborhood Longitude'] 
onehot['isStarbucks'] = (venues['Venue'] == "Starbucks")
# move neighborhood column to the first column
fixed_columns = [onehot.columns[-3]] + [onehot.columns[-2]] + [onehot.columns[-1]] + list(onehot.columns[:-3])
onehot = onehot[fixed_columns]

onehot.isStarbucks = onehot.isStarbucks.astype(int)

onehot.head()
onehot['isStarbucks'].value_counts()

0    1427
1      29
Name: isStarbucks, dtype: int64

In [26]:
venues.head()

,Point Name,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
0,point0,33.868545,-117.75197,Wood Ranch BBQ & Grill,33.866333,-117.751359,252,BBQ Joint
1,point0,33.868545,-117.75197,CAVA,33.867626,-117.749470,252,Mediterranean Restaurant
2,point0,33.868545,-117.75197,Residence Inn Anaheim Hills,33.866789,-117.752782,209,Hotel
3,point0,33.868545,-117.75197,Pavilions,33.867209,-117.751465,155,Grocery Store
4,point0,33.868545,-117.75197,Panera Bread,33.868454,-117.749429,235,Bakery


In [27]:
grouped = onehot.groupby('point_name').mean().reset_index()
grouped

,point_name,point_lat,point_lon,isStarbucks,ATM,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,...,Video Store,Vietnamese Restaurant,Wagashi Place,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,point0,33.868545,-117.751970,0.052632,0.000000,0.000000,0.0000,0.000000,0.105263,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,point1,38.165852,-122.263920,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,point10,34.079716,-118.151899,0.076923,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,point11,37.804200,-122.402488,0.058824,0.000000,0.000000,0.0000,0.000000,0.058824,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,point12,37.768162,-122.402516,0.058824,0.000000,0.000000,0.0000,0.000000,0.058824,0.000000,...,0.000000,0.029412,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412
5,point13,34.160600,-118.757134,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,point14,37.795192,-122.425203,0.000000,0.000000,0.000000,0.0000,0.083333,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,point15,37.885310,-122.307930,0.166667,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,point16,37.786182,-122.425207,0.000000,0.000000,0.000000,0.0000,0.000000,0.071429,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000
9,point17,37.788222,-122.277604,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [28]:
num_top_venues = 5

for point in grouped['point_name']:
    print("----"+point+"----")
    temp = grouped[grouped['point_name'] == point].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----point0----
                    venue  freq
0      Italian Restaurant  0.11
1           Grocery Store  0.11
2        Sushi Restaurant  0.11
3  Furniture / Home Store  0.11
4     American Restaurant  0.11


----point1----
              venue  freq
0              Pool  0.33
1  Basketball Court  0.33
2    Ice Cream Shop  0.33
3       Music Venue  0.00
4         Nightclub  0.00


----point10----
                venue  freq
0          Poke Place  0.08
1   Mobile Phone Shop  0.08
2  Chinese Restaurant  0.08
3         Coffee Shop  0.08
4          Hobby Shop  0.08


----point11----
                  venue  freq
0                Cruise  0.12
1       Coworking Space  0.06
2        Clothing Store  0.06
3    Seafood Restaurant  0.06
4  Gym / Fitness Center  0.06


----point12----
                    venue  freq
0  Furniture / Home Store  0.12
1             isStarbucks  0.06
2     American Restaurant  0.06
3                     Gym  0.06
4             Coffee Shop  0.06


----point13----
        

In [29]:
grouped.head()

,point_name,point_lat,point_lon,isStarbucks,ATM,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,...,Video Store,Vietnamese Restaurant,Wagashi Place,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,point0,33.868545,-117.751970,0.052632,0.0,0.0,0.0,0.0,0.105263,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,point1,38.165852,-122.263920,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,point10,34.079716,-118.151899,0.076923,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,point11,37.804200,-122.402488,0.058824,0.0,0.0,0.0,0.0,0.058824,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,point12,37.768162,-122.402516,0.058824,0.0,0.0,0.0,0.0,0.058824,0.0,...,0.0,0.029412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029412


In [30]:
y = grouped['isStarbucks'] > 0
y = pd.DataFrame(y)
y = y.isStarbucks.astype(int)
print(y.value_counts())
X = grouped
print(X.shape)
X .drop(columns=['point_name', 'point_lat', 'point_lon', 'isStarbucks'], inplace=True)
print(X.columns)
print(X.shape)

0    26
1    23
Name: isStarbucks, dtype: int64
(49, 278)
Index(['ATM', 'Accessories Store', 'Afghan Restaurant', 'African Restaurant',
       'American Restaurant', 'Amphitheater', 'Antique Shop', 'Arcade',
       'Argentinian Restaurant', 'Art Gallery',
       ...
       'Video Store', 'Vietnamese Restaurant', 'Wagashi Place', 'Whisky Bar',
       'Wine Bar', 'Wine Shop', 'Winery', 'Wings Joint', 'Women's Store',
       'Yoga Studio'],
      dtype='object', length=274)
(49, 274)


In [31]:
from sklearn.model_selection import train_test_split
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(39, 274) (39,)
(10, 274) (10,)


In [32]:
pdytrain=pd.DataFrame(y_train)
pdytest=pd.DataFrame(y_test)

pdytrain['isStarbucks'].value_counts()


0    21
1    18
Name: isStarbucks, dtype: int64

In [33]:
pdytest['isStarbucks'].value_counts()

1    5
0    5
Name: isStarbucks, dtype: int64

In [34]:
import sklearn as sk  
from sklearn.ensemble import RandomForestClassifier

#RF = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)  
RF = RandomForestClassifier(n_estimators=100, max_depth=200, random_state=0)  
RF.fit(X_train, y_train)  
RF.predict(X_train)  
RF.score(X_train,y_train)
print("Train accuracy = " + str(RF.score(X_train,y_train)))

Train accuracy = 1.0


In [35]:
pred_test=RF.predict(X_test)  
print("Test accuracy = " + str(RF.score(X_test,y_test)))

Test accuracy = 0.9


In [36]:
# What about AUROC?
prob_y = RF.predict_proba(X)
prob_y
prob_y_train = RF.predict_proba(X_train)
prob_y_test = RF.predict_proba(X_test)


In [37]:
from sklearn.metrics import roc_curve, roc_auc_score
print("Train ROC AUC = " + str(roc_auc_score(y_train, prob_y_train[:,1])))
print("Test ROC AUC = " + str(roc_auc_score(y_test, prob_y_test[:,1])))
#from sklearn.metrics import f1_score
#f1_score(y_test, pred_test, average=None)

Train ROC AUC = 1.0
Test ROC AUC = 0.88


In [38]:
estimator = RF.estimators_[0]

In [39]:
estimator

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=200,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False,
            random_state=209652396, splitter='best')

In [40]:
from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = X.columns,
                class_names = ('near-Starbucks','not-near'),
                rounded = True, proportion = False, 
                precision = 2, filled = True)

In [41]:
# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')